# Speed testing

Purpose of this Notebook is to test the speed of different packages/functions
to achieve a fast simulation process

---


### Networkit vs Netowrkx vs Graph Tool

The purpose is to test which methods from networkx or networkit or graph tool are faster 
to ensure optimal coding experience

In [2]:
"""Import necessary packages."""
import itertools
import os
import sys

import graph_tool.all as gt
import networkit as nk
import networkx as nx
import numpy as np

# Get directory first
path = os.getcwd()
par_dir = os.path.abspath(os.path.join(path, "../"))
# Import own module
sys.path.append(par_dir)
if True:
    from src.network_utils.network_combiner import NetworkCombiner
    from src.network_utils.network_converter import NetworkConverter
    from src.network_utils.network_reader import NetworkReader
    from src.simulators.meta_simulator import MetaSimulator

%load_ext cython

---

Random Graph Speed Test

In [10]:
"""Testing random graph speed."""
n_nodes = 1000
prob = 0.3

print("Networkx:")
%timeit nx.erdos_renyi_graph(n=n_nodes, p=prob)
nk.overview(nk.nxadapter.nx2nk(nx.erdos_renyi_graph(n=n_nodes, p=prob)))

print("\nNetworkit:")
%timeit nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()
nk.overview(nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate())


print("\nGraph Tool:")


def sample_k(max):
    """Sample example."""
    accept = False
    while not accept:
        k = np.random.randint(1, max + 1)
        accept = np.random.random() < 1.0 / k
    return k


%timeit gt.random_graph(N=n_nodes, deg_sampler= lambda: sample_k(40), directed=False)
graph = gt.random_graph(N=n_nodes, deg_sampler=lambda: sample_k(40), directed=False)

Networkx:
134 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Network Properties:
nodes, edges			1000, 150195
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.300691
clustering coefficient		0.300785
min/max/avg degree		257, 343, 300.390000
degree assortativity		-0.005037
number of connected components	1
size of largest component	1000 (100.00 %)

Networkit:
2.45 ms ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Network Properties:
nodes, edges			1000, 149343
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.298985
clustering coefficient		0.299122
min/max/avg degree		249, 341, 298.686000
degree assortativity		0.000588
number of connected components	1
size of largest component	1000 (100.00 %)

Graph Tool:
26.3 ms ± 267 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


---

Barabasi-Albert Speed Test


In [11]:
"""Testing Barabasi-speed test."""
n_nodes = 1000
n_edges = 3

print("Networkx:")
%timeit nx.barabasi_albert_graph(n=n_nodes, m=n_edges)
nk.overview(nk.nxadapter.nx2nk(nx.barabasi_albert_graph(n=n_nodes, m=n_edges)))

print("\nNetworkit:")
%timeit nk.generators.BarabasiAlbertGenerator(k=n_edges,nMax=n_nodes).generate()
nk.overview(nk.generators.BarabasiAlbertGenerator(k=n_edges, nMax=n_nodes).generate())

Networkx:
4.19 ms ± 9.96 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Network Properties:
nodes, edges			1000, 2991
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.005988
clustering coefficient		0.027347
min/max/avg degree		2, 85, 5.982000
degree assortativity		0.304154
number of connected components	1
size of largest component	1000 (100.00 %)

Networkit:
115 µs ± 309 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Network Properties:
nodes, edges			1000, 2994
directed?			False
weighted?			False
isolated nodes			0
self-loops			0
density				0.005994
clustering coefficient		0.032660
min/max/avg degree		3, 97, 5.988000
degree assortativity		0.291063
number of connected components	1
size of largest component	1000 (100.00 %)



---
Coversion Speed Test

In [7]:
"""Testing conversion speed."""
n_nodes = 5000
prob = 0.3

nx_graph = nx.erdos_renyi_graph(n=n_nodes, p=prob)
nk_graph = nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()
gt_graph = NetworkConverter.nx_to_gt(nx_graph)

print("\nNetworkx to Networkit:")
%timeit NetworkConverter.nx_to_nk(nx_graph)

print("\nNetworkit to Networkx:")
%timeit NetworkConverter.nk_to_nx(nk_graph)

print("\nNetworkx to Graph_tool:")
%timeit NetworkConverter.nx_to_gt(nx_graph)

print("\nGraph_tool to Networkx:")
%timeit NetworkConverter.gt_to_nx(gt_graph)

print("\nGraph_tool to Networkit:")
%timeit NetworkConverter.gt_to_nk(gt_graph)

print("\nNeworkit to Graph_tool:")
%timeit NetworkConverter.nk_to_gt(nk_graph)


Networkx to Networkit:
932 ms ± 4.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Networkit to Networkx:
3.34 s ± 36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Networkx to Graph_tool:
14.4 s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Graph_tool to Networkx:


15.2 s ± 97.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Graph_tool to Networkit:
1.34 s ± 4.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Neworkit to Graph_tool:
13.2 s ± 137 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---

Speed Test the different networkit algorithm to each other

---

### Testing the speed of various functions

The purpose is to optimise python in order to have the best results once dealing with intense networks

In [3]:
# create first graph
n_nodes = 1000
prob = 0.1
graph = nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()

nx_graph = NetworkConverter.nk_to_nx(graph)
gt_graph = NetworkConverter.nx_to_gt(nx_graph)

# Inbetweeness

print("\nActuall Betweeness:")
btwn = nk.centrality.Betweenness(graph)
btwn.run()
print(btwn.ranking()[:5])
%timeit btwn.run()

print("\nApproxBetweenness:")
ab = nk.centrality.ApproxBetweenness(graph, epsilon=0.1)
ab.run()
print(ab.ranking()[:5])
%timeit ab.run()

print("\nEstimateBetweenness:")
est = nk.centrality.EstimateBetweenness(graph, 50, True, False)
est.run()
print(est.ranking()[:5])
%timeit est.run()

print("\nKadabraBetweenness:")
kadabra = nk.centrality.KadabraBetweenness(graph, 0.05, 0.8)
kadabra.run()
print(kadabra.ranking()[:5])
%timeit kadabra.run()

print("\nBetweennes graphtool:")
%timeit gt.betweenness(gt_graph)


Actuall Betweeness:
[(649, 1553.3239293741933), (33, 1512.7341964269292), (799, 1511.560738487518), (520, 1444.8862672709363), (609, 1436.119080696174)]
50.2 ms ± 548 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

ApproxBetweenness:
[(596, 0.009280742459396751), (341, 0.0069605568445475635), (346, 0.0069605568445475635), (348, 0.0069605568445475635), (594, 0.0069605568445475635)]
12.6 ms ± 170 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

EstimateBetweenness:
[(575, 0.0028346081860362838), (739, 0.002510685825083123), (862, 0.0025091905178749106), (561, 0.002396430642841699), (584, 0.0023683847499349152)]
40.8 ms ± 716 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

KadabraBetweenness:
[(49, 0.013888888888888888), (83, 0.00992063492063492), (55, 0.00992063492063492), (38, 0.00992063492063492), (416, 0.00992063492063492)]
7.34 ms ± 97 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Betweennes graphtool:
34.9 ms ± 287 µs per loop (mean ± std. dev

Testing if yield is faster than return with the fibonacci sequence

In [14]:
def fib_yield(n):
    """Fibonacci sequence with yield."""
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b


def fib_return(n):
    """Fibonacci sequence with return."""
    a, b = 0, 1
    sequence = []
    for _ in range(n):
        sequence.append(a)
        a, b = b, a + b


n = 100000
print("Fibonacci sequence with yield:")
%timeit list(fib_yield(n))

print("\nFibonacci sequence with return:")
%timeit fib_return(n)

Fibonacci sequence with yield:
170 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Fibonacci sequence with return:
172 ms ± 568 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


---
Testing if IF condition is faster than collect the all the values and get the min value

In [15]:
n_edges = 4
n_nodes = 50
random_network = nk.generators.BarabasiAlbertGenerator(
    k=n_edges, nMax=n_nodes
).generate()


def get_radius(network):
    """Get the radius of a graph which is the minimum eccentricity."""
    # predefine the len of the list for speed
    eccentricity = np.zeros(network.numberOfNodes(), dtype=np.int64)
    # to append to the right idx in the list
    iterator = iter(range(0, network.numberOfNodes()))

    for node in network.iterNodes():
        eccentricity[next(iterator)] = nk.distance.Eccentricity.getValue(network, node)[
            1
        ]

    radius = np.min(eccentricity)
    return int(radius)


def get_radius_slow(network):
    """Get the radius of a graph which is the minimum eccentricity."""
    # predefine the len of the list for speed
    eccentricity = []

    for node in network.iterNodes():
        eccentricity.append(nk.distance.Eccentricity.getValue(network, node)[1])

    radius = min(eccentricity)
    return int(radius)


print("Radius with numpy:")
%timeit get_radius(random_network)
print(f"Radius is {get_radius(random_network)}")

print("\nRadius with predefined list:")
%timeit get_radius_slow(random_network)
print(f"Radius is {get_radius_slow(random_network)}")

Radius with numpy:
182 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Radius is 0

Radius with predefined list:
169 µs ± 3.16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Radius is 0


---
Testing list vs numpy array manipulation

In [1]:
n_nodes = 1000
prob = 3
network = nk.generators.ErdosRenyiGenerator(n_nodes, prob).generate()

apsp = nk.distance.APSP(network)
apsp.run()
# Vector of list for each node to each node
vector_of_dist = apsp.getDistances()

print("\nUsing list comprehension")
%timeit list(itertools.chain.from_iterable(vector_of_dist))
l_list_dist = list(itertools.chain.from_iterable(vector_of_dist))

%timeit list(filter(lambda a: a != 0, l_list_dist))
l_list_dist = list(filter(lambda a: a != 0, l_list_dist))

%timeit list(map(lambda x: 1/x, l_list_dist))
l_inv = list(map(lambda x: 1 / x, l_list_dist))

print("\nUsing numpy comprehension:")
%timeit np.hstack(vector_of_dist)
np_list_dist = np.hstack(vector_of_dist)

%timeit np_list_dist[np_list_dist != 0]
np_list_dist = np_list_dist[np_list_dist != 0]

%timeit np.reciprocal(np_list_dist)
np_inv = np.reciprocal(np_list_dist)


# Compare results
n = network.numberOfNodes()
l_efficiency = (1 / (n * (n - 1))) * np.mean(l_inv)
print("\nList comprehension results :", l_efficiency)
np_efficiency = (1 / (n * (n - 1))) * np.mean(np_inv)
print("\nNumpy comprehension results :", np_efficiency)

NameError: name 'nk' is not defined

---
Test the speed of NetworkCombiner

In [3]:
network = NetworkReader().read_cunha()
gt_network = NetworkConverter.nx_to_gt(network)
%timeit NetworkCombiner.combine_by_preferential_attachment(network,500,2)
%timeit NetworkCombiner.combine_by_preferential_attachment_faster(gt_network,500,2)

15.4 s ± 876 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
241 ms ± 8.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


---
Test the speed of the counting_status_proprotions

In [154]:
import pandas as pd
meta_sim = MetaSimulator(network_name='montagna_calls',ratio_honest=0.9,ratio_wolf=0.08)

def slct_pers_n_neighbours(network,n_groups,network_size):
        """Randomly select the protagonist (person who can act) and its neighbours."""
        communities = {}
        protagonists = np.random.randint(network_size,size=n_groups)
        for protagonist in protagonists:
            communities[protagonist] = [protagonist] + list(network.iter_all_neighbors(protagonist))
        return communities

%timeit slct_pers_n_neighbours(meta_sim.network,200,meta_sim.network.num_vertices())


2.26 ms ± 37.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [155]:
slct_pers_n_neighbours(meta_sim.network,200,meta_sim.network.num_vertices())


{2302: [2302, 1060, 688],
 1940: [1940, 612, 34],
 4002: [4002, 1017, 1323, 4041],
 2381: [2381, 1212, 2097, 2693, 2946],
 322: [322, 10, 34, 1095, 1647, 2454, 2893, 3357],
 2946: [2946, 2381, 34],
 3891: [3891, 1129, 1067],
 696: [696, 411, 157, 918, 3671, 4077, 4557],
 1378: [1378, 25, 108],
 3806: [3806, 1726, 2, 4728],
 796: [796, 223, 731, 1099, 1953, 4208],
 381: [381,
  15,
  300,
  507,
  768,
  980,
  1369,
  2472,
  2743,
  2839,
  3006,
  4356,
  4371,
  4417],
 4594: [4594, 102, 849],
 4370: [4370, 1799, 3575],
 3116: [3116, 956, 844, 4054, 4123],
 3028: [3028, 25, 654, 4034],
 3085: [3085, 2991, 105],
 4541: [4541, 881, 1049],
 3320: [3320, 125, 578, 4620],
 3532: [3532, 1126, 1896],
 4103: [4103, 942, 773],
 3065: [3065, 1827, 1376],
 2299: [2299, 834, 1459, 3944],
 2517: [2517, 978, 2134, 3488],
 2942: [2942, 11, 61],
 1100: [1100, 125, 2, 1164, 1877, 2436, 3836],
 2313: [2313, 2198, 79, 4183, 4550],
 1763: [1763, 1331, 131, 2934],
 3302: [3302, 104, 400],
 4263: [4263, 